In [1]:
import pandas as pd
from SQL import create_db_connection, execute_query
import matplotlib.pyplot as plt
import h5py
import datetime
import numpy as np
import geopandas as gpd
from netCDF4 import Dataset
import os
import xarray as xr
import requests
import netCDF4
import s3fs
import boto3
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import glob
from dateutil import parser


from shapely.geometry import Point, Polygon

#plotting configurations
#plotting preferences
plt.rcParams['font.size'] = '44'

# Import CSBF Telemetry Data

In [2]:
df = pd.read_fwf("/Users/kirahart/Dropbox/Research/balloon/telemetry_data/flight.rpt")

In [3]:
# convert strings to datetime objects

times = np.array('2021-08-30 ' + df['TIME'])
dt = []
for d in range(len(times)):
    v = parser.parse(times[d])
    dt.append(v)

df['datetime'] = dt
df

,TIME,BAR ALT,MBS,AIRT,DRAD,LAT,LON,GPS ALT,datetime
0,15:28:24,4396.0,872.70,22.3,32.7,34-29.4,-104-13.1,1264.0,2021-08-30 15:28:24
1,15:29:24,5509.0,836.03,18.6,30.8,34-29.4,-104-13.2,1507.0,2021-08-30 15:29:24
2,15:30:24,6538.0,803.26,19.1,29.4,34-29.5,-104-13.1,1864.0,2021-08-30 15:30:24
3,15:31:24,7560.0,771.79,16.8,28.5,34-29.7,-104-13.0,2228.0,2021-08-30 15:31:24
4,15:32:24,8528.0,742.91,15.1,27.2,34-29.9,-104-12.9,2523.0,2021-08-30 15:32:24
...,...,...,...,...,...,...,...,...,...
274,20:02:47,14255.0,589.76,5.5,22.4,34-30.8,-105-53.0,4506.0,2021-08-30 20:02:47
275,20:03:47,13190.0,616.04,6.5,24.4,34-30.5,-105-53.1,4159.0,2021-08-30 20:03:47
276,20:04:47,12188.0,641.67,7.6,25.1,34-30.2,-105-53.0,3836.0,2021-08-30 20:04:47
277,20:05:47,11013.0,672.82,10.3,26.0,34-30.1,-105-53.0,3602.0,2021-08-30 20:05:47


In [4]:
def convert(string):
    nums = string.split('-')
    if len(nums) >2 :
        nums = nums[1:]
        val = '-'+nums[0]+'.'+nums[1][-4:-2] +nums[1][-1]
    else:
        val = nums[0]+'.'+nums[1][-4:-2] +nums[1][-1]
    return(float(val))
    
df['LAT'] = df['LAT'].apply(lambda x: convert(x))
df['LON'] = df['LON'].apply(lambda x: convert(x))

In [5]:
geometry = [Point(xy) for xy in zip(df["LON"],df['LAT'])]

In [6]:
df['geometry'] = geometry
crs = {'init':'epsg:4326'}

In [7]:
geo_df = gpd.GeoDataFrame(df,geometry = geometry)

# Load GOES Data

In [8]:
def get_GEOS_data(bucket_name,product,year,day_of_year,hour,band):
    #construct file path to AWS bucket
    fp_bucket =bucket_name+product+'/'+str(year)+'/'+str(day_of_year) +'/'+str(hour)
    f = fp_bucket  +'/OR_'+product+'-M6C'+str(band)
    print(f) 

    # Use the anonymous credentials to access public data
    fs = s3fs.S3FileSystem(anon=True)

    # List contents of GOES-16 bucket.
    all_bin = fs.ls(fp_bucket)
    print(fp_bucket)

    # Download the first file, and rename it the same name (without the directory structure)
    for name in all_bin:
        fn = '/Users/kirahart/Dropbox/Research/balloon/telemetry_data/GOES/' + name.split('/')[-1]
        fs.get(name,fn)

In [9]:
bucket_name = 'noaa-goes16/'
product = 'ABI-L2-ACHTF'
year = 2021
day_of_year = 242
band = 13

In [10]:
#QUERY AWS bucket for desired hours
for hour in [15,16,17,18,19]:
    print(hour)
    get_GEOS_data(bucket_name,product,year,day_of_year,hour,band)

15
noaa-goes16/ABI-L2-ACHTF/2021/242/15/OR_ABI-L2-ACHTF-M6C13
noaa-goes16/ABI-L2-ACHTF/2021/242/15
16
noaa-goes16/ABI-L2-ACHTF/2021/242/16/OR_ABI-L2-ACHTF-M6C13
noaa-goes16/ABI-L2-ACHTF/2021/242/16
17
noaa-goes16/ABI-L2-ACHTF/2021/242/17/OR_ABI-L2-ACHTF-M6C13
noaa-goes16/ABI-L2-ACHTF/2021/242/17
18
noaa-goes16/ABI-L2-ACHTF/2021/242/18/OR_ABI-L2-ACHTF-M6C13
noaa-goes16/ABI-L2-ACHTF/2021/242/18
19
noaa-goes16/ABI-L2-ACHTF/2021/242/19/OR_ABI-L2-ACHTF-M6C13
noaa-goes16/ABI-L2-ACHTF/2021/242/19


In [11]:
#from mpl_toolkits.basemap import Basemap, cm

def lat_lon_reproj(file):
    g16_data_file = file # select .nc file
    
    # designate dataset
    g16nc = Dataset(g16_data_file, 'r')
    var_names = [ii for ii in g16nc.variables]
    var_name = var_names[0]
    try:
        band_id = g16nc.variables['band_id'][:]
        band_id = ' (Band: {},'.format(band_id[0])
        band_wavelength = g16nc.variables['band_wavelength']
        band_wavelength_units = band_wavelength.units
        band_wavelength_units = '{}'.format(band_wavelength_units)
        band_wavelength = ' {0:.2f} '.format(band_wavelength[:][0])
        
    except:
        band_id = ''
        band_wavelength = ''
        band_wavelength_units = ''

    # GOES-R projection info and retrieving relevant constants
    proj_info = g16nc.variables['goes_imager_projection']
    lon_origin = proj_info.longitude_of_projection_origin
    H = proj_info.perspective_point_height+proj_info.semi_major_axis
    r_eq = proj_info.semi_major_axis
    r_pol = proj_info.semi_minor_axis

    # grid info
    lat_rad_1d = g16nc.variables['x'][:]
    lon_rad_1d = g16nc.variables['y'][:]

    # data info    
    data = g16nc.variables[var_name][:]
    data_units = g16nc.variables[var_name].units
    data_time_grab = ((g16nc.time_coverage_end).replace('T',' ')).replace('Z','')
    data_long_name = g16nc.variables[var_name].long_name
    
    # close file when finished
    g16nc.close()
    g16nc = None

    # create meshgrid filled with radian angles
    lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)

    # lat/lon calc routine from satellite radian angle vectors

    lambda_0 = (lon_origin*np.pi)/180.0

    a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*(np.power(np.cos(lon_rad),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(lon_rad),2.0))))
    b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
    c_var = (H**2.0)-(r_eq**2.0)

    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)

    s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
    s_y = - r_s*np.sin(lat_rad)
    s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)

    lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)

    return lon,lat,data,data_units,data_time_grab,data_long_name,band_id,band_wavelength,band_wavelength_units,var_name



In [12]:
fn = '/Users/kirahart/Dropbox/Research/balloon/telemetry_data/GOES/'
ext = 'OR_'+product+'*'
files = glob.glob(fn+ext)

results = []
for file in files:
    result = lat_lon_reproj(file)
    results.append(result)
    
cnames = ['lon','lat','data','data_units','time','name','band_id','band_wavelength','wavelength_units','var_name']
GOES16 = pd.DataFrame(results, columns = cnames)

/Users/kirahart/opt/anaconda3/envs/database/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in sqrt
/Users/kirahart/opt/anaconda3/envs/database/lib/python3.7/site-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in sqrt


In [13]:
def bandplot(goes,telem,i,cmax,name):
    d = goes.loc[i]
    #find balloon closest position
    dt = parser.parse(d['time'])
    i = np.argmin(np.abs(telem.datetime - dt))
    current_lat = telem.iloc[i]['LAT']
    current_lon = telem.iloc[i]['LON']
    
    fig = plt.figure(figsize=(25, 15), dpi=100)
    ax = plt.gca()
    ax.set_facecolor("lightgrey")
    plt.pcolormesh(d['lon'].data,d['lat'].data,d['data'], cmap = "plasma",shading = 'auto')
    

    plt.xlim(-106.5,-103)
    plt.ylim(33.5,35)


    plt.colorbar(label = '\n ' + d['name'] + ' [' + d['data_units'] + ']')
    plt.clim(200,300)

    plt.plot(geo_df['LON'],geo_df['LAT'],'w--',linewidth=6)
    plt.plot(current_lon, current_lat,'k.',markersize = 60)
    circle2 = plt.Circle((current_lon, current_lat), 0.2, color='k', fill=False,linewidth=8)
    ax.add_patch(circle2)


    plt.ylabel("Latitude [$\circ$]")
    plt.xlabel("Longitude [$\circ$]")
    plt.locator_params(nbins=4)
    plt.title( d['time']+' Z')
    
    fig.savefig(name)
    plt.close()

In [14]:
#MAKE SURE THE GIF IS GENERATED IN CHRON ORDER
GOES16['Date'] = pd.to_datetime(GOES16.time)
G12 = GOES16.sort_values('Date')
order = G12.index.values.tolist()

In [15]:
for i in range(28):
    count = order[i]
    fn= '/Users/kirahart/Dropbox/Research/balloon/telemetry_data/figures/GOES16/CLOUD/'
    if i < 10:
        name = fn + '0'+ str(i)+'.png'
    else:
        name = fn + str(i)+'.png'
    figure = bandplot(GOES16,df,count,100,name)
    
    count = count + 1

/Users/kirahart/opt/anaconda3/envs/database/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  if sys.path[0] == '':
/Users/kirahart/opt/anaconda3/envs/database/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  if sys.path[0] == '':
/Users/kirahart/opt/anaconda3/envs/database/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorre

/Users/kirahart/opt/anaconda3/envs/database/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  if sys.path[0] == '':
/Users/kirahart/opt/anaconda3/envs/database/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  if sys.path[0] == '':
/Users/kirahart/opt/anaconda3/envs/database/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorre

In [16]:
from PIL import Image

# filepaths
fp_in = '/Users/kirahart/Dropbox/Research/balloon/telemetry_data/figures/GOES16/CLOUD/*.png'
fp_out ='/Users/kirahart/Dropbox/Research/balloon/telemetry_data/figures/GOES16/CLOUD/CLOUD.gif'
# https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=600, loop=0)
